In [1]:
import numpy as np
import torch
from torch.autograd import Variable
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import datasets
import torch.nn.functional as F
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import copy
# from art.attacks.poisoning import *



device=0

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

transform_train = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Lambda(lambda x: F.pad(
            Variable(x.unsqueeze(0), requires_grad=False),
            (4, 4, 4, 4), mode='reflect').data.squeeze()),
        transforms.ToPILImage(),
        transforms.RandomCrop(32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

batch_size = 2048

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, #batch_size=batch_size,
                                         )

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, #batch_size=batch_size,
                                        )
                                         #shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
import torch.nn as nn
import torch.nn.functional as F

net = torchvision.models.resnet18(pretrained=True)

net.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(512, 10)
)
net = net.to(device)
# params (LR) recommended in paper
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

/home/dezhang/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dezhang/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
import os
import torchvision.transforms as T
directory = os.fsencode("poisoned_cifar10/train/cat/")
poisoned_imgs = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".png"): 
        a=torchvision.io.read_image("poisoned_cifar10/train/cat/"+filename)
        b=T.ToPILImage()(a) 
        poisoned_imgs.append(transform(b))
        continue
    else:
        continue
poisoned_labels = torch.Tensor([3 for i in poisoned_imgs]).to(device)
poisoned_imgs = torch.stack(poisoned_imgs).to(device)

In [5]:
import os
import torchvision.transforms as T
directory = os.fsencode("poisoned_cifar10/targets/cat/")
poisoned_imgs_test = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".png"): 
        a=torchvision.io.read_image("poisoned_cifar10/targets/cat/"+filename)
        b=T.ToPILImage()(a) 
        poisoned_imgs_test.append(transform(b))
        continue
    else:
        continue
poisoned_labels_test = torch.Tensor([1 for i in poisoned_imgs_test]).to(device)
poisoned_imgs_test = torch.stack(poisoned_imgs_test*2).to(device)

In [6]:
poisoned_imgs_test.shape

torch.Size([2, 3, 32, 32])

In [7]:
x_train = torch.Tensor(np.array([i[0][0] for i in trainloader])).to(device)
y_train = torch.Tensor(np.array([i[1][0] for i in trainloader])).type(torch.long).to(device)

In [13]:
import copy

n_epoch = 100
batch_size = 2048
for _ in range(n_epoch):
    print("epoch " + str(_))
    for i in range(0,len(x_train_poisoned),batch_size):

        # net_copy = copy.deepcopy(net)
        # feature = x_train_poisoned[i:i+batch_size]
        # feature.requires_grad = True  ### CRUCIAL LINE !!!
        # target = y_train_poisoned[i:i+batch_size]
        # optimizer.zero_grad()
        # output = net_copy(feature)
        # loss = criterion(output, target)
        # loss.backward()
        # optimizer.step()

        # # removing the outliers
        # kept_idx = randomized_agg_forced(feature.grad.flatten(start_dim=-1))
        # # print(kept_idx.shape)
        # kept_idx = [i for i in range(len(kept_idx)) if kept_idx[i]]

        
        feature = x_train_poisoned[i:i+batch_size]
        target = y_train_poisoned[i:i+batch_size]
        optimizer.zero_grad()
        output = net(feature)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        pred_class_poison = torch.argmax(net(poisoned_imgs_test), dim=-1)
        #adding the [0] here because for some reason torch wont accept a single input. 
        #poisoned_imgs_test contains two copies of the poisoned image
        # print(torch.argmax(net(poisoned_imgs_test), dim=-1))
        if 3 in pred_class_poison:
            print("attack succeed")
        else:
            print("attack fail")
        train_acc = torch.sum(torch.argmax(net(x_train), dim=-1) == y_train)/len(y_train)
        test_acc = torch.sum(torch.argmax(net(x_test), dim=-1) == y_test)/len(y_test)
        print("TRAIN ACC: ", float(train_acc), "TEST ACC: ", float(test_acc))

epoch 0
attack succeed
TRAIN ACC:  0.12783999741077423 TEST ACC:  0.12700000405311584
epoch 1
attack fail
TRAIN ACC:  0.15495999157428741 TEST ACC:  0.16099999845027924
epoch 2
attack fail
TRAIN ACC:  0.18361999094486237 TEST ACC:  0.1842999905347824
epoch 3
attack fail
TRAIN ACC:  0.20965999364852905 TEST ACC:  0.2150999903678894
epoch 4
attack fail
TRAIN ACC:  0.23283998668193817 TEST ACC:  0.2345999926328659
epoch 5
attack fail
TRAIN ACC:  0.2549000084400177 TEST ACC:  0.25689998269081116
epoch 6
attack succeed
TRAIN ACC:  0.27636000514030457 TEST ACC:  0.289000004529953
epoch 7
attack fail
TRAIN ACC:  0.29523998498916626 TEST ACC:  0.29839998483657837
epoch 8
attack fail
TRAIN ACC:  0.30952000617980957 TEST ACC:  0.3130999803543091
epoch 9
attack fail
TRAIN ACC:  0.3245999813079834 TEST ACC:  0.3271999955177307
epoch 10
attack fail
TRAIN ACC:  0.3372199833393097 TEST ACC:  0.3443000018596649
epoch 11
attack fail
TRAIN ACC:  0.35308000445365906 TEST ACC:  0.3489999771118164
epoch 12

In [14]:
poisoned_imgs_test.shape

torch.Size([2, 3, 32, 32])